In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import sys
%matplotlib inline
import scipy.stats as st
import pprint
# Import API key
g_key = ""
# Configure API key
gmaps.configure(api_key=g_key)

#Data File path (CSV)
data = "./output_data/latlng.csv"
data2 = "./output_data/zip_grp.csv"

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Study data files
RMS_path = "./RMS.csv"
# Read the data
Crime_df = pd.read_csv(RMS_path, dtype={"ZIP":object})
print(Crime_df)


       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
2         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219220  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219221  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219223  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [3]:
#Clean the Zipcode Data
Crime_df = Crime_df.dropna(subset=['ZIP'])
Crime_df = Crime_df.loc[(Crime_df["OFFENSE-DESC"]!="Other Crimes")]
print(Crime_df)

       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
5         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219216  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219217  12/31/2018  12/31/2018  2018  DECEMBER    53   F    W      H  NaN   
219218  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [4]:
Crime_df=Crime_df.loc[(Crime_df["ZIP"]!="0") & (Crime_df["ZIP"]!="1") & (Crime_df["ZIP"]!="78") & 
                                (Crime_df["ZIP"]!="11111") & (Crime_df["ZIP"]!="6278") & (Crime_df["ZIP"]!="50701")
                               & (Crime_df["ZIP"]!="81507") & (Crime_df["ZIP"]!="70205") & (Crime_df["ZIP"]!="78200")
                     & (Crime_df["ZIP"]!="78000")]
print(Crime_df)

       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
5         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219216  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219217  12/31/2018  12/31/2018  2018  DECEMBER    53   F    W      H  NaN   
219218  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [5]:
# of zipcodes
Zipcodes_df = Crime_df.loc[:,["ZIP"]]
print(Zipcodes_df)
print(type(Zipcodes_df))
Zipcodes_df["ZIP"].unique()

          ZIP
0       78205
1       78201
3       78201
4       78207
5       78207
...       ...
219216  78250
219217  78223
219218  78220
219219  78220
219222  78257

[136123 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


array(['78205', '78201', '78207', '78210', '78230', '78212', '78228',
       '78242', '78221', '78238', '78213', '78202', '78245', '78233',
       '78219', '78224', '78240', '78251', '78216', '78237', '78209',
       '78227', '78218', '78254', '78211', '78223', '78252', '78220',
       '78266', '78247', '78232', '78214', '78204', '78256', '78250',
       '78229', '78249', '78217', '78235', '78203', '78215', '78258',
       '78253', '78023', '78239', '78225', '78226', '78244', '78222',
       '78259', '78255', '78130', '78248', '78208', '78257', '78260',
       '78231', '78109', '78070', '78234', '78264', '78056', '78006',
       '78261', '78241', '78155', '78108', '78132', '78148', '78073',
       '78112', '78236', '78015', '78025', '78154', '78648', '78163',
       '78121', '78288', '78069'], dtype=object)

In [6]:
Num_of_Crimes_grp = Crime_df.groupby("ZIP")
Num_of_Crimes = Num_of_Crimes_grp["INCIDENT_NUM"].count()
Num_of_Crimes_2 = Num_of_Crimes.astype(float)
Num_of_Crimes_grp2 = Crime_df.groupby("OFFENSE-DESC")
Num_of_Crimes_3 = Num_of_Crimes_grp2["ZIP"].count()
Num_of_Crimes_4 = pd.DataFrame(Num_of_Crimes_3)
print(type(Num_of_Crimes_4))

print(Num_of_Crimes_4)

<class 'pandas.core.frame.DataFrame'>
                                     ZIP
OFFENSE-DESC                            
Aggravated Assault                  1367
Aggravated Family Violence           896
Aggravated Robbery Business          164
Aggravated Robbery Individual        443
Aggravated Sexual Assault             37
Arson                                 75
Assault                             2477
Burglary Building                    337
Burglary Habitation                  412
Burglary Vehicle                     694
Capital Murder                        14
Deadly Conduct                       623
Driving While Intoxicated           1864
Drug Arrest                        39457
Family Violence                     7834
Fraud                               1494
Graffiti                             118
Indecency with a Child                42
Indecency/Obscenity                   12
Indecent Exposure                    196
Injury to Elderly Person             257
Injury to a Child  

In [7]:
Zipcodes_df2 = pd.DataFrame(Zipcodes_df["ZIP"].unique(), columns=["Zip"])

Zipcodes_df2["Lat"] = ""
Zipcodes_df2["Lon"] = ""

print(Zipcodes_df2.dtypes)
print(Zipcodes_df2)

Zip    object
Lat    object
Lon    object
dtype: object
      Zip Lat Lon
0   78205        
1   78201        
2   78207        
3   78210        
4   78230        
..    ...  ..  ..
75  78648        
76  78163        
77  78121        
78  78288        
79  78069        

[80 rows x 3 columns]


In [8]:
#create for loop to gather lat and long for zipcodes to apply to heatmap
for index,row in Zipcodes_df2.iterrows():

    # get zipcode info from df
    Zipcode = row['Zip']

    # print results index and zipcode
    print(f"Retrieving Results for Index {index}: {Zipcode}.")
    # Build the endpoint URL and make API request
    target_url = ("https://maps.googleapis.com/maps/api/place/textsearch/json?query="+str(Zipcode)+"&key="+g_key)
    response = requests.get(target_url).json()
    pprint.pprint(response)
    
    # extract results
    results = response['results']
    
    
    try:
        print(f"The lat is: {results[0]['geometry']['location']['lat']}")
        print(f"The lng is: {results[0]['geometry']['location']['lng']}")
        
        Zipcodes_df2.loc[index, "Lat"] = response['results'][0]['geometry']['location']['lat']
        Zipcodes_df2.loc[index, "Lon"] = response['results'][0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Zipcodes_df2.to_csv(data)

Retrieving Results for Index 0: 78205.
{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78205, USA',
              'geometry': {'location': {'lat': 29.422658,
                                        'lng': -98.48699049999999},
                           'viewport': {'northeast': {'lat': 29.436588,
                                                      'lng': -98.468075},
                                        'southwest': {'lat': 29.4128519,
                                                      'lng': -98.497957}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78205',
              'place_id': 'ChIJGfaPt6pYXIYRjGJaIKQdkeQ',
              'reference': 'ChIJGfaPt6pYXIYRjGJaIKQdkeQ',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.422658
The lng is: -98.48699049999999
------------
Retrieving Results for Index 1: 78201.
{'html_attributions': [

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78238, USA',
              'geometry': {'location': {'lat': 29.4687013, 'lng': -98.6215649},
                           'viewport': {'northeast': {'lat': 29.518958,
                                                      'lng': -98.5861001},
                                        'southwest': {'lat': 29.43611289999999,
                                                      'lng': -98.638218}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78238',
              'place_id': 'ChIJB_khFwddXIYRlacupKVWb6E',
              'reference': 'ChIJB_khFwddXIYRlacupKVWb6E',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.4687013
The lng is: -98.6215649
------------
Retrieving Results for Index 10: 78213.
{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78213, USA',
              

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78216, USA',
              'geometry': {'location': {'lat': 29.5516834,
                                        'lng': -98.49522280000001},
                           'viewport': {'northeast': {'lat': 29.5831949,
                                                      'lng': -98.4377469},
                                        'southwest': {'lat': 29.487988,
                                                      'lng': -98.529477}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78216',
              'place_id': 'ChIJ1X53th5gXIYRG9Io4xbwvSY',
              'reference': 'ChIJ1X53th5gXIYRG9Io4xbwvSY',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.5516834
The lng is: -98.49522280000001
------------
Retrieving Results for Index 19: 78237.
{'html_attributions': [],
 'results': [{'formatted_address

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78220, USA',
              'geometry': {'location': {'lat': 29.4143476,
                                        'lng': -98.39101269999999},
                           'viewport': {'northeast': {'lat': 29.4446799,
                                                      'lng': -98.33372},
                                        'southwest': {'lat': 29.389158,
                                                      'lng': -98.446383}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78220',
              'place_id': 'ChIJX_r9jE_xXIYRkAZdOVzhJxc',
              'reference': 'ChIJX_r9jE_xXIYRkAZdOVzhJxc',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.4143476
The lng is: -98.39101269999999
------------
Retrieving Results for Index 28: 78266.
{'html_attributions': [],
 'results': [{'formatted_address':

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78249, USA',
              'geometry': {'location': {'lat': 29.5676137, 'lng': -98.6050736},
                           'viewport': {'northeast': {'lat': 29.5984488,
                                                      'lng': -98.5523549},
                                        'southwest': {'lat': 29.541477,
                                                      'lng': -98.6658655}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78249',
              'place_id': 'ChIJYfryL-5gXIYRs-J4zdeHvLI',
              'reference': 'ChIJYfryL-5gXIYRs-J4zdeHvLI',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.5676137
The lng is: -98.6050736
------------
Retrieving Results for Index 37: 78217.
{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78217, USA',
              'geome

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78225, USA',
              'geometry': {'location': {'lat': 29.385634, 'lng': -98.5254159},
                           'viewport': {'northeast': {'lat': 29.398214,
                                                      'lng': -98.511144},
                                        'southwest': {'lat': 29.3784899,
                                                      'lng': -98.5510949}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78225',
              'place_id': 'ChIJhYzDrOJYXIYR0B11mWmnaRY',
              'reference': 'ChIJhYzDrOJYXIYR0B11mWmnaRY',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.385634
The lng is: -98.5254159
------------
Retrieving Results for Index 46: 78226.
{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78226, USA',
              'geometry

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78260, USA',
              'geometry': {'location': {'lat': 29.6897433,
                                        'lng': -98.49522280000001},
                           'viewport': {'northeast': {'lat': 29.7428559,
                                                      'lng': -98.44269279999999},
                                        'southwest': {'lat': 29.6342621,
                                                      'lng': -98.564686}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78260',
              'place_id': 'ChIJERemhvmHXIYRUKVhmfhOEPs',
              'reference': 'ChIJERemhvmHXIYRUKVhmfhOEPs',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.6897433
The lng is: -98.49522280000001
------------
Retrieving Results for Index 56: 78231.
{'html_attributions': [],
 'results': [{'formatted

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78241, USA',
              'geometry': {'location': {'lat': 29.39, 'lng': -98.58},
                           'viewport': {'northeast': {'lat': 29.39972149999999,
                                                      'lng': -98.56399259999999},
                                        'southwest': {'lat': 29.3802776,
                                                      'lng': -98.59600739999999}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78241',
              'place_id': 'ChIJhf-nnNtbXIYR0W2lxEkCgoY',
              'reference': 'ChIJhf-nnNtbXIYR0W2lxEkCgoY',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.39
The lng is: -98.58
------------
Retrieving Results for Index 65: 78155.
{'html_attributions': [],
 'results': [{'formatted_address': 'Seguin, TX 78155, USA',
              'geometry

{'html_attributions': [],
 'results': [{'formatted_address': 'Ingram, TX 78025, USA',
              'geometry': {'location': {'lat': 30.0934313, 'lng': -99.3118282},
                           'viewport': {'northeast': {'lat': 30.166284,
                                                      'lng': -99.212693},
                                        'southwest': {'lat': 29.995696,
                                                      'lng': -99.3680669}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'Ingram, TX 78025',
              'place_id': 'ChIJnUqBNoNZWYYR1H6r0bS3qao',
              'reference': 'ChIJnUqBNoNZWYYR1H6r0bS3qao',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 30.0934313
The lng is: -99.3118282
------------
Retrieving Results for Index 74: 78154.
{'html_attributions': [],
 'results': [{'formatted_address': 'Selma, TX 78154, USA',
              'geometry': {'location':

In [9]:
# convert zipcodes to floats
Zipcodes_df2["Lat"] = pd.to_numeric(Zipcodes_df2["Lat"])
Zipcodes_df2["Lon"] = pd.to_numeric(Zipcodes_df2["Lon"])

# locations and weight for use in our heatmap
locations = Zipcodes_df2[["Lat", "Lon"]]
Crime = Num_of_Crimes_2
print(len(locations))
print(len(Crime))

80
80


In [10]:
#create heatmap
fig= gmaps.figure(center=[29.50,-98.50], zoom_level=11)
heat_layer = gmaps.heatmap_layer(locations, weights=Crime, dissipating=False,)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
#create csv so pivot chart can be made
Data_A = Crime_df.groupby(["ZIP"])['OFFENSE-DESC'].value_counts()
print(Data_A)
Crime_grp = pd.DataFrame(Data_A)
Crime_grp.to_csv(data2)


ZIP    OFFENSE-DESC                     
78006  TABC-Liquor Law Violation             2
78015  TABC-Liquor Law Violation             1
78023  Drug Arrest                          46
       TABC-Liquor Law Violation            29
       Family Violence                      14
                                            ..
78266  Unauthorized Use of Motor Vehicle     1
78288  Drug Arrest                           2
       Driving While Intoxicated             1
       Weapon Violation                      1
78648  Panhandling                           1
Name: OFFENSE-DESC, Length: 1877, dtype: int64
